In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from analysis import *
import plotly.offline as offline

This notebook compares sensitivities of individual variables, changing one-at-a-time (OAT). The rest of the variables remain as per baseline.

A positive `delta` indicates that buying would result in a financial gain over renting after 5 years.

## Baseline

In [2]:
print_baseline()

Simulation Duration                    5 years
Deposit                            30000 £
Initial Property Value            200000 £
Property Value Yearly Increase     0.025 %
Mortgage Term                         25 years
Mortgage Interest Rate              0.05 %
Initial Monthly Rent Price          1200 £
Rent Price Yearly Increase          0.03 %
Savings Interest Rate               0.05 %
Delta                              56008 £


In [3]:
vars = [
    ("initialPropertyValue", "£", 2),
    ("propertyValueYearlyIncrease", "%", 1),
    ("mortgageInterestRate", "%", 1),
    ("initialMonthlyRentPrice", "£", 3),
    ("rentPriceYearlyIncrease", "%", 1),
    ("savingsInterestRate", "%", 1),
]


def scatter(var, units, slope=False):
    df = load_csv(var)
    y = df["delta"] if slope == False else np.gradient(df["delta"])
    units_prefix = units if units == "£" else ""
    units_suffix = units if units == "%" else ""
    return go.Scatter(
        x=df[var] * infer_multiplier(units),
        y=y,
        name=get_axis_title(var),
        mode="markers+lines",
        hovertemplate=f"{units_prefix}%{{x}}{units_suffix}, £%{{y}}",
    )

In [33]:
fig = make_subplots(cols=3, horizontal_spacing=0.01)
for name, units, col in vars:
    fig.add_trace(scatter(name, units), row=1, col=col)
fig.add_hline(
    56000,
    line_color="green",
    line_dash="dash",
    line_width=0.5,
    annotation_text="Baseline",
)
fig.add_hline(0, line_color="black", line_width=2)
y_range = [-100_000, 250_000]
fig.update_layout(
    xaxis1={"title_text": "Value (%)"},
    xaxis2={"title_text": get_axis_title("initialPropertyValue")},
    xaxis3={"title_text": get_axis_title("initialMonthlyRentPrice")},
    yaxis1={"range": y_range, "title_text": "Delta (£)"},
    yaxis2={"range": y_range, "showticklabels": False},
    yaxis3={"range": y_range, "showticklabels": False},
    legend={"xanchor": "center", "x": 0.5, "y": -0.2, "orientation": "h"},
    font={"size": 8},
    margin={"l": 0, "r": 0, "t": 0, "b": 0},
)
fig.show()

offline.plot(fig, filename='Presentation/plots/1d-sensitivity.html', auto_open=False)


'Presentation/plots/1d-sensitivity.html'

In [36]:
fig = make_subplots(cols=3, horizontal_spacing=0.01)
for name, units, col in vars:
    fig.add_trace(scatter(name, units, slope=True), row=1, col=col)
y_range = [-20_000, 20_000]
fig.update_layout(
    xaxis1={"title_text": "Value (%)"},
    xaxis2={"title_text": get_axis_title("initialPropertyValue")},
    xaxis3={"title_text": get_axis_title("initialMonthlyRentPrice")},
    yaxis1={"range": y_range, "title_text": "Slope"},
    yaxis2={"range": y_range, "showticklabels": False},
    yaxis3={"range": y_range, "showticklabels": False},
    legend={"xanchor": "center", "x": 0.5, "y": -0.2, "orientation": "h"},
    font={"size": 8},
    margin={"l": 0, "r": 0, "t": 0, "b": 0},
) 
fig.show()
offline.plot(fig, filename='Presentation/plots/1d-sensitivity-slope.html', auto_open=False)

'Presentation/plots/1d-sensitivity-slope.html'